# 张量

In [1]:
%reload_ext mermaid

<IPython.core.display.Javascript object>

TF全称叫做TensorFlow，tensor是贯穿TF的最重要的数据结构。它本质上是一个对高维数据的封装，提供了丰富的API。在线性代数中，我们常用向量、矩阵来表示数据，而在深度学习应用中，有对更高维数据的需求。比如在对图像进行处理时，彩色图像本身就带有三维的信息（长、宽、颜色通道），通常还需要对彩色图像进行批处理，这样待处理的数据变为四维，在一些特殊的情形下，往往还需要更高维度的数据。如果针对每种多维数据定义一种结构，必然给计算带来不便。TF的做法是，为高维数据定义统一的类型Tensor。

参见[tensor.h](../tensorflow/core/framework/tensor.h)的Tensor数据成员的定义

```c++

class Tensor {
    //...
private:
    TensorShape shape_;
    TensorBuffer* buf_;
}
```

`TensorShape` 和`TensorBuffer`是两个辅助的类，分别用来管理Tensor的形状的存储Tensor的实际数据。

Tensor作为一个核心数据类，必然提供了很多API，比如常规的构造、析构、赋值、复制、数值属性获取等。除此之外，特别关注Tensor的转化接口

```c++
class Tensor {
  public:
    //...
    //与proto数据的相互转化
    bool FromProto(const TensorProto& other);
    void AsProtoField(TensorProto* proto);
    //为底层数据创建新视图
    template <typename T> typename TTypes<T>::Vec vec();
    template <typename T> typename TTypes<T>::Matrix matrix();
    template <typename T> typename TTypes<T, NDIMS>::Tensor tensor();
}

```

其中第一类将Tensor与序列化的proto之间相互转化，便于在设备之间传递Tensor。第二类是为当前的Tensor的底层数据提供另外一种视图。

回顾Tensor包含的私有数据，TensorBuffer* buffer_是一个指向底层数据的指针，关于它的结构在下文中会详细说明。这意味着，Tensor并不包含实际的底层数据，它实际上只是对底层数据的一种视图。同样一份底层数据，可以提供多种视图。比如对于一个长度为12的数组，如果把它看做向量，它是一个1x12的向量，如果把它看作矩阵，可以认为是3x4或者2x6的矩阵，如果把它当作张量，可以认为是3x2x2的张量。通过这种方法，我们可以对同一份底层数据进行复用，避免了重复申请内存空间，提升了效率。实际上，numpy的底层多多维数组的实现，也是同样的道理。

```mermaid
graph TB
    A("Tensor A, shape=[3,4]")-->D("底层数据TensorBuffer")
    B("Tensor B, shape=[2,6]")-->D("底层数据TensorBuffer")
    C("Tensor B, shape=[3,2,2]")-->D("底层数据TensorBuffer")
```
